In [ ]:
import sys
sys.path.append('/home/fjargsto/AcousticBEM/Python')

from InteriorHelmholtzSolver2D import *
from InteriorHelmholtzSolver2D_C import *

c         = 344.0 # speed of sound [m/s]
rho       = 1.205 # density of air [kg/m^3]
frequency = 400.0 # frequency [Hz]

print "c   = ", c
print "rho = ", rho
print "f   = ", frequency

useNative = False

In [ ]:
aVertex = np.array([[0.00, 0.0000], [0.00, 0.0125], [0.00, 0.0250], [0.00, 0.0375],
                     [0.00, 0.0500], [0.00, 0.0625], [0.00, 0.0750], [0.00, 0.0875],
                     
                     [0.0000, 0.10], [0.0125, 0.10], [0.0250, 0.10], [0.0375, 0.10],
                     [0.0500, 0.10], [0.0625, 0.10], [0.0750, 0.10], [0.0875, 0.10],
                     
                     [0.10, 0.1000], [0.10, 0.0875], [0.10, 0.0750], [0.10, 0.0625],
                     [0.10, 0.0500], [0.10, 0.0375], [0.10, 0.0250], [0.10, 0.0125],
                     
                     [0.1000, 0.00], [0.0875, 0.00], [0.0750, 0.00], [0.0625, 0.00],
                     [0.0500, 0.00], [0.0375, 0.00], [0.0250, 0.00], [0.0125, 0.00]], dtype=np.float32)

aEdge = np.array([[ 0,  1], [ 1,  2], [ 2,  3], [ 3,  4], 
                  [ 4,  5], [ 5,  6], [ 6,  7], [ 7,  8],
                  
                  [ 8,  9], [ 9, 10], [10, 11], [11, 12],
                  [12, 13], [13, 14], [14, 15], [15, 16],
                  
                  [16, 17], [17, 18], [18, 19], [19, 20],
                  [20, 21], [21, 22], [22, 23], [23, 24],
                  
                  [24, 25], [25, 26], [26, 27], [27, 28],
                  [28, 29], [29, 30], [30, 31], [31,  0]], dtype=np.int32)

interiorPoints = np.array([[0.0250, 0.0250],
                           [0.0750, 0.0250],
                           [0.0250, 0.0750],
                           [0.0750, 0.0750],
                           [0.0500, 0.0500]], dtype=np.float32)

In [ ]:
centers = 0.5 * (aVertex[aEdge[:, 0]] + aVertex[aEdge[:, 1]])

In [ ]:
# Test Problem 1
# Dirichlet boundary condition with phi = sin(k/sqrt(2)*x) * sin(k/sqrt(2)*y)
#
if useNative:
    solver = InteriorHelmholtzSolver2D_C(aVertex, aEdge)
else:
     solver = InteriorHelmholtzSolver2D(aVertex, aEdge)
   

boundaryCondition = BoundaryCondition(aEdge.shape[0])
boundaryCondition.alpha.fill(1.0)
boundaryCondition.beta.fill(0.0)
boundaryCondition.f

k = solver.frequencyToWavenumber(frequency)

boundaryCondition.f[:] = np.sin(k/np.sqrt(2.0) * centers[:,0]) \
                       * np.sin(k/np.sqrt(2.0) * centers[:,1])

boundaryIncidence = BoundaryIncidence(aEdge.shape[0])
boundaryIncidence.phi.fill(0.0)
boundaryIncidence.v.fill(0.0)

interiorIncidentPhi = np.zeros(interiorPoints.shape[0], dtype=np.complex64)

boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)
interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)
print "Test Problem 1"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)


# Test Problem 2
# von Neumann boundary condition such that phi = sin(k/sqrt(2) * x) * sin(k/sqrt(2) * y)
# Differentiate with respect to x and y to obtain outward normal:
# dPhi/dX = k/sqrt(2) * cos(k/sqrt(2) * x) * sin(k/sqrt(2) * y)
# dPhi/dY = k/sqrt(2) * sin(k/sqrt(2) * x) * cos(k/sqrt(2) * y)
boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)
c = k / np.sqrt(2.0)
for i in range(centers.shape[0]):
    x = centers[i, 0]
    y = centers[i, 1]
    if (x < 1e-7):
        boundaryCondition.f[i] = -c * np.cos(c * x) * np.sin(c * y)
    elif (x > 0.1 - 1e-7):
        boundaryCondition.f[i] =  c * np.cos(c * x) * np.sin(c * y)
    elif (y < 1e-7):
        boundaryCondition.f[i] = -c * np.sin(c * x) * np.cos(c * y)
    else:
        boundaryCondition.f[i] =  c * np.sin(c * x) * np.cos(c * y)        

boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)
interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)
print "\n\nTest Problem 2"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)

   
# Test Problem 3
# The test problem computes the field produced by a unit source at
# the point (0.5,0.25) within the square with a rigid boundary.
# The rigid boundary implies the bondary condition v=0.
# The test problem computes the field produced by a unit source at
# the point (0.5,0.25) within the square with a rigid boundary.
# The incident velocity potential is given by {\phi}_inc=i*h0(kr)/4
# where r is the distance from the point (0.5,0.25)
boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)
boundaryCondition.f.fill(0.0)

p = np.array([0.05, 0.025], dtype=np.float32)
for i in range(centers.shape[0]):
    r = centers[i] - p
    R = norm(r)
    boundaryIncidence.phi[i] = 0.25j * hankel1(0, k * R)
    if centers[i, 0] < 1e-7:
        boundaryIncidence.v[i] = -0.25j * k * hankel1(1, k * R) * (-r[0] / R)
    elif centers[i, 0] > 0.1 - 1e-7:
        boundaryIncidence.v[i] = -0.25j * k * hankel1(1, k * R) * ( r[0] / R)
    elif centers[i, 1] < 1e-7:
        boundaryIncidence.v[i] = -0.25j * k * hankel1(1, k * R) * (-r[1] / R)
    elif centers[i, 1] > 0.1 - 1e-7:
        boundaryIncidence.v[i] = -0.25j * k * hankel1(1, k * R) * ( r[1] / R)
    else:
        assert False, "All cases must be handled above."
        
for i in range(interiorIncidentPhi.size):
    r = interiorPoints[i] - p
    R = norm(r)
    interiorIncidentPhi[i] = 0.25j * hankel1(0, k * R)
       
boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)
interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)
print "\n\nTest Problem 3"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)
 